Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

# Regularization on Tensorflow model

In [7]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
    biases = tf.Variable(tf.zeros([1024]))
    
    # Computing logit 
    logits = tf.matmul(tf_train_dataset, weights) + biases

    # Adding hidden Layer 
    hidden_nodes = 1024
    hidden_weights = tf.Variable( tf.truncated_normal([hidden_nodes, num_labels]) )
    hidden_biases = tf.Variable( tf.zeros([num_labels]))
    hidden_layer = tf.nn.relu(tf.matmul(logits, hidden_weights) + hidden_biases)
    
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(hidden_layer, tf_train_labels) + \
                        0.1*tf.nn.l2_loss(hidden_weights) +
                        0.1*tf.nn.l2_loss(hidden_biases) +
                        0.1*tf.nn.l2_loss(weights) +
                        0.1*tf.nn.l2_loss(biases))
          
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
    # Testing performance. 
    train_prediction = tf.nn.softmax(hidden_layer)
    
    valid_relu = tf.nn.relu(  tf.matmul(tf_valid_dataset, weights) + biases)
    valid_prediction = tf.nn.softmax( tf.matmul(valid_relu, hidden_weights) + hidden_biases) 

    test_relu = tf.nn.relu( tf.matmul( tf_test_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu, hidden_weights) + hidden_biases)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 31684.095703
Minibatch accuracy: 15.6%
Validation accuracy: 23.6%
Test accuracy: 25.9%
Minibatch loss at step 500: 2.634185
Minibatch accuracy: 84.4%
Validation accuracy: 79.5%
Test accuracy: 86.8%
Minibatch loss at step 1000: 1.396455
Minibatch accuracy: 76.6%
Validation accuracy: 78.9%
Test accuracy: 85.9%
Minibatch loss at step 1500: 1.214907
Minibatch accuracy: 83.6%
Validation accuracy: 79.5%
Test accuracy: 86.7%
Minibatch loss at step 2000: 1.283907
Minibatch accuracy: 85.9%
Validation accuracy: 78.8%
Test accuracy: 86.2%
Minibatch loss at step 2500: 1.442972
Minibatch accuracy: 75.8%
Validation accuracy: 79.1%
Test accuracy: 86.1%
Minibatch loss at step 3000: 1.449243
Minibatch accuracy: 78.1%
Validation accuracy: 79.0%
Test accuracy: 85.9%


# Let's test regularization on logistic regression model

In [9]:
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    labels = save['train_labels']
    t_labels = save['test_labels']
    v_labels = save['valid_labels']
    del save

In [10]:
X = train_dataset.reshape(train_dataset.shape[0], -1)[:5000, :]
y = labels[:5000]

In [11]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty="l2")
clf.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [12]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

pred_test = clf.predict(test_dataset.reshape(test_dataset.shape[0], -1))
valid_test = clf.predict(valid_dataset.reshape(valid_dataset.shape[0], -1))

# Accuracy of the classifier. 
print(accuracy_score(t_labels, pred_test))
print(accuracy_score(v_labels, valid_test))

# F1 score of the classifier. 
print(f1_score(t_labels, pred_test, average='macro'))
print(f1_score(v_labels, valid_test, average='macro'))

0.8511
0.7757
0.851249136465
0.775767438061


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [13]:
num_steps = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 31798.478516
Minibatch accuracy: 12.5%
Validation accuracy: 20.4%
Test accuracy: 21.6%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [14]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
    biases = tf.Variable(tf.zeros([1024]))
    
    # Computing logit 
    logits = tf.matmul(tf_train_dataset, weights) + biases

    # Adding hidden Layer 
    hidden_nodes = 1024
    hidden_weights = tf.Variable( tf.truncated_normal([hidden_nodes, num_labels]) )
    hidden_biases = tf.Variable( tf.zeros([num_labels]))
    hidden_layer = tf.nn.relu(tf.matmul(tf.nn.dropout(logits, keep_prob=0.9), hidden_weights) + hidden_biases)
    
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(hidden_layer, tf_train_labels) + \
                        0.1*tf.nn.l2_loss(hidden_weights) +
                        0.1*tf.nn.l2_loss(hidden_biases) +
                        0.1*tf.nn.l2_loss(weights) +
                        0.1*tf.nn.l2_loss(biases))
          
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
    # Testing performance. 
    train_prediction = tf.nn.softmax(hidden_layer)
    
    valid_relu = tf.nn.relu(  tf.matmul(tf_valid_dataset, weights) + biases)
    valid_prediction = tf.nn.softmax( tf.matmul(valid_relu, hidden_weights) + hidden_biases) 

    test_relu = tf.nn.relu( tf.matmul( tf_test_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu, hidden_weights) + hidden_biases)

In [15]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 31873.324219
Minibatch accuracy: 11.7%
Validation accuracy: 20.2%
Test accuracy: 20.7%
Minibatch loss at step 500: 2.634774
Minibatch accuracy: 84.4%
Validation accuracy: 80.1%
Test accuracy: 87.1%
Minibatch loss at step 1000: 1.400337
Minibatch accuracy: 77.3%
Validation accuracy: 79.2%
Test accuracy: 86.3%
Minibatch loss at step 1500: 1.216217
Minibatch accuracy: 84.4%
Validation accuracy: 80.0%
Test accuracy: 86.9%
Minibatch loss at step 2000: 1.287921
Minibatch accuracy: 85.9%
Validation accuracy: 79.5%
Test accuracy: 86.8%
Minibatch loss at step 2500: 1.441922
Minibatch accuracy: 75.0%
Validation accuracy: 79.4%
Test accuracy: 86.1%
Minibatch loss at step 3000: 1.448254
Minibatch accuracy: 78.1%
Validation accuracy: 79.7%
Test accuracy: 86.5%


Overfitting case With Dropout 

In [121]:
num_steps = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 31877.156250
Minibatch accuracy: 7.8%
Validation accuracy: 16.9%
Test accuracy: 18.2%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


Using the normal initialized making the loss function "nan"

In [17]:
def init_weights(shape, init_method='xavier', xavier_params = (None, None)):
    if init_method == 'zeros':
        return tf.Variable(tf.zeros(shape, dtype=tf.float32))
    elif init_method == 'uniform':
        return tf.Variable(tf.random_normal(shape, stddev=0.01, dtype=tf.float32))
    else: #xavier
        (fan_in, fan_out) = xavier_params
        low = -4*np.sqrt(6.0/(fan_in + fan_out)) # {sigmoid:4, tanh:1} 
        high = 4*np.sqrt(6.0/(fan_in + fan_out))
        return tf.Variable(tf.random_uniform(shape, minval=low, maxval=high, dtype=tf.float32))

In [29]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = init_weights([image_size * image_size, 1024], \
                                   xavier_params=[image_size * image_size, 1024])
    biases = init_weights([1024], init_method="zeros")
    
    # Computing logit 
    logits = tf.matmul(tf_train_dataset, weights) + biases

    # Adding hidden Layer 1
    hidden_nodes = 1024
    keep_prob = tf.placeholder(tf.float32)
    
    hidden_weights1 = init_weights([hidden_nodes, hidden_nodes], \
                                   xavier_params=[hidden_nodes, hidden_nodes])
    hidden_biases1 = tf.Variable( tf.zeros([hidden_nodes]))
    hidden_layer1 = tf.nn.relu(tf.matmul(logits, hidden_weights1) + hidden_biases1)
    
    # Adding hidden Layer 2    
    hidden_weights2 = init_weights([hidden_nodes, num_labels], \
                                   xavier_params=[hidden_nodes, num_labels])
    hidden_biases2 = tf.Variable( tf.zeros([num_labels]))
    hidden_layer2 = tf.nn.relu(tf.matmul(hidden_layer1, hidden_weights2) + hidden_biases2)
    
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(hidden_layer2, tf_train_labels) + \
                        0.001*tf.nn.l2_loss(hidden_weights1) +
                        0.001*tf.nn.l2_loss(hidden_biases1) +
                        0.001*tf.nn.l2_loss(hidden_weights2) +
                        0.001*tf.nn.l2_loss(hidden_biases2) +
                        0.001*tf.nn.l2_loss(weights) +
                        0.001*tf.nn.l2_loss(biases))
          
    # Optimizer.
    #global_step = tf.Variable(0)  # count the number of steps taken.
    #learning_rate = tf.train.exponential_decay(0.5, global_step, batch_size, 0.9)
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Testing performance. 
    train_prediction = tf.nn.softmax(hidden_layer2)
    
    valid_relu = tf.nn.relu(  tf.matmul(tf_valid_dataset, weights) + biases)
    valid_prediction1 = tf.nn.relu( tf.matmul(valid_relu, hidden_weights1) + hidden_biases1) 
    valid_prediction = tf.nn.softmax( tf.matmul(valid_prediction1, hidden_weights2) + hidden_biases2) 
    
    test_relu = tf.matmul( tf_test_dataset, weights) + biases
    test_prediction1 = tf.nn.relu(tf.matmul(test_relu, hidden_weights1) + hidden_biases1)
    test_prediction2 = tf.nn.relu(tf.matmul(test_prediction1, hidden_weights2) + hidden_biases2)
    test_prediction = tf.nn.softmax(test_prediction2)

In [30]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 50.041058
Minibatch accuracy: 10.9%
Validation accuracy: 18.2%
Test accuracy: 21.6%
Minibatch loss at step 500: 51795327357235398443008.000000
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Test accuracy: 10.0%
Minibatch loss at step 1000: 31411542299597336477696.000000
Minibatch accuracy: 8.6%
Validation accuracy: 10.0%
Test accuracy: 10.0%
Minibatch loss at step 1500: 19049683740022099935232.000000
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
Test accuracy: 10.0%
Minibatch loss at step 2000: 11552766620319803768832.000000
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Test accuracy: 10.0%
Minibatch loss at step 2500: 7006231362951817723904.000000
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Test accuracy: 10.0%
Minibatch loss at step 3000: 4248963852639154470912.000000
Minibatch accuracy: 7.8%
Validation accuracy: 10.0%
Test accuracy: 10.0%
